In [4]:
import tensorflow as tf

import tensorflow.keras as keras 
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Data Loading & Preprocessing

In [5]:
data_train = pd.read_csv('./data/kaggle/train_split.csv')
data_valid = pd.read_csv('./data/kaggle/val_split.csv')
data_train['date'] = pd.to_datetime(data_train['date'])
data_train['date']=data_train['date'].map(dt.datetime.toordinal)
data_valid['date'] = pd.to_datetime(data_valid['date'])
data_valid['date']=data_valid['date'].map(dt.datetime.toordinal)
y_train = data_train['sales']
X_train = data_train.drop('sales', axis=1)
y_valid = data_valid['sales']
X_valid = data_valid.drop('sales', axis=1)
X_train = X_train.values
y_train = y_train.values
X_valid = X_valid.values
y_valid = y_valid.values
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)

# Deep Learning Neural Network Initialization & Training

In [6]:
model = keras.models.Sequential()
model.add(keras.layers.Input((3,)))
model.add(keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(1, activation='relu', kernel_initializer='he_normal'))

In [7]:
checkpoint_filepath = './model/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.01),
    loss=tf.keras.losses.MeanAbsoluteError(),
    metrics=[tf.keras.metrics.MeanAbsoluteError()],
)

In [21]:
# model.fit(X_train, y_train, epochs=20, batch_size=1024, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

Train on 730400 samples, validate on 182600 samples
Epoch 1/20
730400/730400 [==============================] - 12s 16us/sample - loss: 8.3504 - mean_absolute_error: 8.3504 - val_loss: 9.2912 - val_mean_absolute_error: 9.2912
Epoch 2/20
730400/730400 [==============================] - 10s 14us/sample - loss: 8.3263 - mean_absolute_error: 8.3263 - val_loss: 8.6607 - val_mean_absolute_error: 8.6607
Epoch 3/20
730400/730400 [==============================] - 10s 14us/sample - loss: 8.3139 - mean_absolute_error: 8.3139 - val_loss: 9.1546 - val_mean_absolute_error: 9.1546
Epoch 4/20
730400/730400 [==============================] - 9s 12us/sample - loss: 8.3149 - mean_absolute_error: 8.3149 - val_loss: 8.3216 - val_mean_absolute_error: 8.3216
Epoch 5/20
730400/730400 [==============================] - 11s 15us/sample - loss: 8.3071 - mean_absolute_error: 8.3071 - val_loss: 8.4798 - val_mean_absolute_error: 8.4798
Epoch 6/20
730400/730400 [==============================] - 11s 15us/sample - l

In [10]:
model = keras.models.load_model('./model/kaggle')

In [11]:
model.evaluate(X_valid, y_valid)

182600/182600 [==============================] - 16s 86us/sample - loss: 8.1861 - mean_absolute_error: 8.1861


[8.186106846507512, 8.1861]

# Plotting Data

In [12]:
def draw(scaler, model, name=None, store=1, item=1, df_path = './data/kaggle/val_split.csv', save_fig=0):
    
    df = pd.read_csv(df_path)
    df = df[(df['item'] == item) & (df['store'] == store)]
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by=['date'], inplace=True, ascending=False)
    dates = df['date'].values
    df['date']=df['date'].map(dt.datetime.toordinal)
    y = df['sales'].values
    df = df.drop('sales', axis=1)
    X = df.values
    X = scaler.transform(X)
    pred = model.predict(X)
    loss, _ = model.evaluate(X, y)
    pred_upper = pred+loss
    pred_lower = pred-loss
#     plt.xlabel(dates)
    plt.rcParams['figure.figsize'] = [200,100]
    plt.rcParams.update({'font.size':200})
    pred_upper = np.reshape(pred_upper, (-1,))
    pred_lower = np.reshape(pred_lower, (-1,))
    plt.plot(dates,y, 'ro', ms=40)
    plt.plot(dates, pred, lw=30)
    plt.plot(dates, pred_upper, lw=10, color='b')
    plt.plot(dates, pred_lower, lw=10, color='b')
    plt.fill_between(dates,pred_upper, pred_lower, color='b', alpha=0.2)
    string = 'store_{}_item_{}'.format(store, item)
    plt.title(string)
    
    
    if save_fig==0:
        plt.show()
    else:
        plt.savefig('./kaggle_data_plots/' + name, dpi=10, format='jpeg')
        plt.clf()
    

In [13]:
draw(scaler=scaler, model=model, store=10, item=50)

373/373 [==============================] - 0s 49us/sample - loss: 10.6143 - mean_absolute_error: 10.6143


In [266]:
# for i in range(10):
#     for j in range(50):
#         draw(save_fig=1, scaler=scaler, model=model, store=i+1, item=j+1, name='store_'+ str(i+1)+'_'+ 'item_' + str(j+1))

391/391 [==============================] - 0s 49us/sample - loss: 12.0394 - mean_absolute_error: 12.0394


348/348 [==============================] - 0s 47us/sample - loss: 3.9370 - mean_absolute_error: 3.9370


368/368 [==============================] - 0s 50us/sample - loss: 11.2463 - mean_absolute_error: 11.2463


373/373 [==============================] - 0s 44us/sample - loss: 10.6143 - mean_absolute_error: 10.6143


<Figure size 14400x7200 with 0 Axes>